In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import json
import os
import re
from collections import Counter

INPUT_PATH  = "/content/drive/MyDrive/dijalekti/texts_clean.jsonl"
OUTPUT_PATH = "/content/drive/MyDrive/dijalekti/texts_segmented.jsonl"

print("Вчитуваме оригинални текстови од:", INPUT_PATH)

records = []
with open(INPUT_PATH, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        text = (obj.get("text") or "").strip()
        dialect = (obj.get("dialect") or "").strip()
        if text != "" and dialect != "":
            records.append((text, dialect))

print("Оригинален број текстови:", len(records))

def split_into_chunks(text,
                      min_words=10,
                      max_words=60):
    """
    1) Го дели текстот на реченици
    2) Ги групира речениците во сегменти со должина помеѓу min_words и max_words
    """

    sentences = re.split(r'(?<=[\.\?\!])\s+|\n+', text)
    sentences = [s.strip() for s in sentences if s.strip() != ""]

    chunks = []
    current = []
    current_len = 0

    for sent in sentences:
        words = sent.split()
        n = len(words)

        if n >= max_words:
            if current_len >= min_words:
                chunks.append(" ".join(current))
                current = []
                current_len = 0
            chunks.append(sent)
            continue

        if current_len + n > max_words and current_len >= min_words:
            chunks.append(" ".join(current))
            current = []
            current_len = 0

        current.append(sent)
        current_len += n

    if current_len >= min_words:
        chunks.append(" ".join(current))

    return chunks

segmented = []
for text, dialect in records:
    chunks = split_into_chunks(text,
                               min_words=10,
                               max_words=60)
    for ch in chunks:
        segmented.append({
            "text": ch,
            "dialect": dialect
        })

print("Број сегменти по сечење:", len(segmented))

cnt = Counter([r["dialect"] for r in segmented])
print("Број сегменти по дијалект:")
for d, c in cnt.items():
    print(f"{d}: {c}")

with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    for obj in segmented:
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")

print("Новиот сегментиран датасет е снимен во:", OUTPUT_PATH)


In [ ]:
import json
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

SEGMENTED_PATH = "/content/drive/MyDrive/dijalekti/texts_segmented.jsonl"

texts = []
dialects = []

with open(SEGMENTED_PATH, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        t = (obj.get("text") or "").strip()
        d = (obj.get("dialect") or "").strip()
        if t != "" and d != "":
            texts.append(t)
            dialects.append(d)

print("Вкупно сегменти:", len(texts))

cnt = Counter(dialects)
print("Број сегменти по дијалект (пред филтрирање):")
for d, c in cnt.items():
    print(f"{d}: {c}")

# За да избегнеме проблеми со класи со 1 пример,
# ги ставаме ретките во 'other'
MIN_SAMPLES = 5
dialects_norm = []
for d in dialects:
    if cnt[d] < MIN_SAMPLES:
        dialects_norm.append("other")
    else:
        dialects_norm.append(d)

cnt2 = Counter(dialects_norm)
print("\nПо нормализација (ретките во 'other'):")
for d, c in cnt2.items():
    print(f"{d}: {c}")

le_seg = LabelEncoder()
y_all = le_seg.fit_transform(dialects_norm)

X_train, X_temp, y_train, y_temp = train_test_split(
    texts, y_all, test_size=0.3, random_state=42, stratify=y_all
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

print("Train:", len(X_train), "Val:", len(X_val), "Test:", len(X_test))
print("Класи:", le_seg.classes_)


Вкупно сегменти: 4975
Број сегменти по дијалект (пред филтрирање):
тетовски (долнополошки): 192
скопскоцрногорски: 49
кумановски: 771
кривопаланечки: 173
овчеполски: 7
кратовски: 93
скопски-велешки: 158
кичевско-поречки: 350
прилепско-битолски: 626
гостиварски (горнополошки): 94
галички: 21
дебарски: 40
вевчанско-радошки: 25
струшки: 519
охридски: 128
горнопреспански: 54
долнопреспански: 38
тиквешко-мариовски: 248
штипско-кочански: 228
малешевско-пирински: 397
гевгелиско-дојрански: 485
струмичко-радовишки: 182
дримколско-голобрдски: 97

По нормализација (ретките во 'other'):
тетовски (долнополошки): 192
скопскоцрногорски: 49
кумановски: 771
кривопаланечки: 173
овчеполски: 7
кратовски: 93
скопски-велешки: 158
кичевско-поречки: 350
прилепско-битолски: 626
гостиварски (горнополошки): 94
галички: 21
дебарски: 40
вевчанско-радошки: 25
струшки: 519
охридски: 128
горнопреспански: 54
долнопреспански: 38
тиквешко-мариовски: 248
штипско-кочански: 228
малешевско-пирински: 397
гевгелиско-дојрански

In [ ]:
text_model_seg = Pipeline([
    ("tfidf", TfidfVectorizer(max_features=7000,
                              ngram_range=(1,2))),  # 1- и 2-gram
    ("clf", LogisticRegression(max_iter=1500, n_jobs=-1))
])

text_model_seg.fit(X_train, y_train)

# Валидација
y_val_pred = text_model_seg.predict(X_val)
print("=== SEGMENTED TEXT MODEL – Validation ===")
print(classification_report(
    y_val, y_val_pred,
    labels=list(range(len(le_seg.classes_))),
    target_names=le_seg.classes_,
    zero_division=0
))

# Тест
y_test_pred = text_model_seg.predict(X_test)
print("=== SEGMENTED TEXT MODEL – Test ===")
print(classification_report(
    y_test, y_test_pred,
    labels=list(range(len(le_seg.classes_))),
    target_names=le_seg.classes_,
    zero_division=0
))


=== SEGMENTED TEXT MODEL – Validation ===
                            precision    recall  f1-score   support

         вевчанско-радошки       0.00      0.00      0.00         5
                   галички       0.00      0.00      0.00         3
      гевгелиско-дојрански       0.86      0.93      0.89        73
           горнопреспански       0.67      0.33      0.44         6
гостиварски (горнополошки)       0.89      0.44      0.59        18
                  дебарски       0.00      0.00      0.00         5
           долнопреспански       0.00      0.00      0.00         7
     дримколско-голобрдски       0.60      0.40      0.48        15
          кичевско-поречки       0.78      0.85      0.82        55
                 кратовски       1.00      0.17      0.29        18
            кривопаланечки       0.83      0.79      0.81        24
                кумановски       0.78      0.98      0.87       132
       малешевско-пирински       0.69      0.78      0.73        59
     